<a href="https://colab.research.google.com/github/juliatessler/1s2023-unicamp-dl-for-search-systems/blob/main/5-training-a-lm/5_train_a_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5- Train a Language Model
by Júlia Tessler

I chose to use the code from GPT4 that was presented in class as a starting point for this exercise. But I also got great ideas from Mirelle's notebook!

## Notebook preparation


In [1]:
!pip install transformers datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
BASE_PATH = '/content/drive/MyDrive/Unicamp/'

# Change this line if you'd like to use a sample of the dataset for debugging reasons
should_use_full_dataset = True

In [ ]:
!gsutil cp gs://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt .

Copying gs://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

/
Operation completed over 1 objects/1.2 GiB.                                      


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Mar 29 11:54:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    49W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [34]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [35]:
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    pipeline
)

## Dataset preparation

In [ ]:
# Creating smaller test files for agility in debuging
!sed -n '1,100p' /content/sample-1gb.txt > /content/small_sample.txt

In [ ]:
if should_use_full_dataset:
  data_file = '/content/sample-1gb.txt'
  print('Using full dataset')
else:
  data_file = '/content/small_sample.txt'
  print('Using small dataset')

output_dir = BASE_PATH+'trained_out/'

dataset = load_dataset("text", data_files = data_file, split = 'train')
dataset = dataset.train_test_split(test_size = 0.20, seed = 42) # separando 20% do dataset para teste

dataset

Using full dataset


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-23a2a74ee1148000/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 200000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 50000
    })
})

In [ ]:
dataset['train']['text'][0]

'As informações sobre a descarga de grãos no Porto de Paranaguá chegarão, esta semana, às rodovias do interior de São Paulo. Os informativos da Operação Safra serão divulgados nas praças de pedágio nos municípios de Caiuá e Presidente Bernardes, na Rodovia Raposo Tavares (SP-270). Os pontos são estratégicos para atingir os caminhoneiros vindos do Mato Grosso, segundo Estado de origem dos grãos descarregados em Paranaguá. De acordo com o setor de estatísticas da Administração dos Portos de Paranaguá e Antonina (Appa), dos 65.087 caminhões que descarregaram grãos no Porto de Paranaguá este ano, até o dia 20 de março, 14.911 eram provenientes do Mato Grosso. No total, os veículos carregavam mais de 546 mil toneladas. O principal produto transportado no período foi soja: quase 273 mil toneladas. Segundo o superintendente da Appa, Luiz Henrique Dividino, o porto paranaense recebeu do Mato Grosso, também, farelo de soja (quase 140 mil toneladas) e milho (mais de 133 mil toneladas). "Depois d

In [ ]:
model_name = 'facebook/opt-125m'

tokenizer = AutoTokenizer.from_pretrained(model_name)

config = AutoConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_config(config)

tokenized_dataset = dataset.map(lambda x : tokenizer(x['text'], truncation = True, padding = "max_length", max_length = 256),
                                batched = True,
                                num_proc = 4,
                                remove_columns = ['text'])


Map (num_proc=4):   0%|          | 0/200000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = 10,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    logging_strategy = "epoch",
    learning_rate = 5e-4,
    weight_decay = 0.01,
    fp16=True
)

data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

trainer = Trainer(
   model = model,
   args = training_args,
   train_dataset = tokenized_dataset['train'],
   eval_dataset = tokenized_dataset['test'],
   data_collator = data_collator
)


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,4.197900,4.066578
2,4.456300,4.498810
3,4.451700,4.185144
4,4.361000,4.111216
5,4.269500,4.040729
6,4.123100,3.968261
7,4.045200,3.851737
8,3.903100,3.729180
9,3.803000,3.663114
10,3.738700,3.630288


TrainOutput(global_step=125000, training_loss=4.1349525625, metrics={'train_runtime': 12354.3842, 'train_samples_per_second': 161.886, 'train_steps_per_second': 10.118, 'total_flos': 2.61292032e+17, 'train_loss': 4.1349525625, 'epoch': 10.0})

In [ ]:
eval_results = trainer.evaluate( eval_dataset=tokenized_dataset['train'])


In [ ]:
perplexity = torch.exp(torch.tensor(eval_results['eval_loss']))
print(f'Perplexity :{perplexity.item()}')

Perplexity :36.780052185058594


In [ ]:
output_dir = BASE_PATH+'trained_out_fromscratch/'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/Unicamp/trained_out_fromscratch/tokenizer_config.json',
 '/content/drive/MyDrive/Unicamp/trained_out_fromscratch/special_tokens_map.json',
 '/content/drive/MyDrive/Unicamp/trained_out_fromscratch/vocab.json',
 '/content/drive/MyDrive/Unicamp/trained_out_fromscratch/merges.txt',
 '/content/drive/MyDrive/Unicamp/trained_out_fromscratch/added_tokens.json',
 '/content/drive/MyDrive/Unicamp/trained_out_fromscratch/tokenizer.json')

## What about fine tuning?

In [ ]:
model_name = 'facebook/opt-125m'

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenized_dataset = dataset.map(lambda x : tokenizer(x['text'], truncation = True, padding = "max_length", max_length = 256),
                                batched = True,
                                num_proc = 4,
                                remove_columns = ['text'])

Map (num_proc=4):   0%|          | 0/200000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = 10,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    logging_strategy = "epoch",
    learning_rate = 5e-4,
    weight_decay = 0.01,
    fp16=True
)

data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

trainer = Trainer(
   model = model,
   args = training_args,
   train_dataset = tokenized_dataset['train'],
   eval_dataset = tokenized_dataset['test'],
   data_collator = data_collator
)


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.713800,2.456354
2,2.371400,2.334416
3,2.237600,2.274769
4,2.141200,2.236405
5,2.059700,2.216497
6,1.984900,2.205371
7,1.912800,2.203961
8,1.841100,2.211366


Epoch,Training Loss,Validation Loss
1,2.713800,2.456354
2,2.371400,2.334416
3,2.237600,2.274769
4,2.141200,2.236405
5,2.059700,2.216497
6,1.984900,2.205371
7,1.912800,2.203961
8,1.841100,2.211366
9,1.770800,2.226111
10,1.706600,2.244669


TrainOutput(global_step=125000, training_loss=2.07397453125, metrics={'train_runtime': 12366.7851, 'train_samples_per_second': 161.724, 'train_steps_per_second': 10.108, 'total_flos': 2.61292032e+17, 'train_loss': 2.07397453125, 'epoch': 10.0})

In [ ]:
eval_results = trainer.evaluate( eval_dataset=tokenized_dataset['test'])

In [ ]:
perplexity = torch.exp(torch.tensor(eval_results['eval_loss']))
print(f'Perplexity :{perplexity.item()}')

Perplexity :9.43729305267334


In [ ]:
output_dir = BASE_PATH+'trained_out_finetuned/'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/Unicamp/trained_out_finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/Unicamp/trained_out_finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/Unicamp/trained_out_finetuned/vocab.json',
 '/content/drive/MyDrive/Unicamp/trained_out_finetuned/merges.txt',
 '/content/drive/MyDrive/Unicamp/trained_out_finetuned/added_tokens.json',
 '/content/drive/MyDrive/Unicamp/trained_out_finetuned/tokenizer.json')

## Qualitative analysis

In [36]:
fromscratch_dir = BASE_PATH + "trained_out_fromscratch/"
model_zero = AutoModelForCausalLM.from_pretrained(fromscratch_dir)
tokenizer_zero = AutoTokenizer.from_pretrained(fromscratch_dir)

In [37]:
finetuned_dir = BASE_PATH+'trained_out_finetuned/'
model_finetuned = AutoModelForCausalLM.from_pretrained(finetuned_dir)
tokenizer_finetuned = AutoTokenizer.from_pretrained(finetuned_dir)

In [44]:
pipe_zero = pipeline(
    "text-generation", model = model_zero, tokenizer = tokenizer_zero, device = 'cuda:0'
)

pipe_ft = pipeline(
    "text-generation", model = model_finetuned, tokenizer = tokenizer_finetuned, device = 'cuda:0'
)

In [49]:
txt_0 = "Era uma vez "

In [50]:
pipe_zero(txt_0)

[{'generated_text': 'Era uma vez irmã, o que é um dos mais importantes do'}]

In [51]:
pipe_ft(txt_0)

[{'generated_text': 'Era uma vez ia dar um tempo pra criar um blog, e'}]

In [52]:
txt_1 = "Fortaleza, clube de glória " # e tradição - é o hino do Fortaleza esporte clube

In [53]:
pipe_zero(txt_1)

[{'generated_text': 'Fortaleza, clube de glória irmãs, ou seja,'}]

In [54]:
pipe_ft(txt_1)

[{'generated_text': 'Fortaleza, clube de glória !! O Fortaleza é um clube'}]